In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sys

sys.path.append('/content/gdrive/My Drive/Colab Notebooks/ECE_Project')

In [0]:
import CNN_Lib
import CNN_BuildingBlock_Lib as BB
import RNN_BuildingBlock_Lib as RNN_BB
import CombinationModel as ComboCNN
import CombinationModel_RNN_CNN as RNN_CNN
import CombinationModel_LSTM_FCN as LSTM_FCN
import numpy as np 
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import random
import pdb
import copy


In [4]:
dir = '/content/gdrive/My Drive/Colab Notebooks/ECE_Project'
X_test = np.load(dir + '/X_test.npy')
y_test = np.load(dir + '/y_test.npy')
person_train_valid = np.load(dir + '/person_train_valid.npy')
X_train_valid = np.load(dir + '/X_train_valid.npy')
y_train_valid = np.load(dir + '/y_train_valid.npy')
person_test = np.load(dir + '/person_test.npy')
print ('Training/Valid data shape: {}' .format(X_train_valid.shape))
print ('Test data shape: {}' .format(X_test.shape))
print ('Training/Valid target shape: {}' .format(y_train_valid.shape))
print ('Test target shape: {}' .format(y_test.shape))
print ('Person train/valid shape: {}' .format(person_train_valid.shape))
print ('Person test shape: {}' .format(person_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [0]:
#transform data into torch-readable data types
#2115 number of trials from 9 people
#four possible classes of outputs
#Xtrain = Variable(torch.from_numpy(X_train_valid)).cuda()
#ytrain = Variable(torch.from_numpy(y_train_valid)).cuda()
#Xtest = Variable(torch.from_numpy(X_test)).cuda()
#ytest = Variable(torch.from_numpy(y_test)).cuda()

Xtrain = torch.from_numpy(X_train_valid)
Ytrain = torch.from_numpy(y_train_valid)
Xtest = torch.from_numpy(X_test)
Ytest = torch.from_numpy(y_test)



In [0]:
def map_to_class(input_labels):
  mask1 = (input_labels == 769)*0
  mask2 = (input_labels == 770)*1
  mask3 = (input_labels == 771)*2
  mask4 = (input_labels == 772)*3

  return (mask1 + mask2 + mask3 + mask4)

In [7]:
########## INITIALIZE LSTM-FCN COMBINATION MODEL ########## 

Trials, InputDim, SeqDim = X_train_valid.shape
num_features = InputDim
Tests = y_test.shape


#-----RNN tweak here----#
hidden_dim = 32 #above 150 and we get low training accuracy
num_layers = 1 #increasing this makes training take MUCH longer, similar validation and lower training accuracy
nonlinearity = 'tanh'
initialization = 'xavierNorm'


#-----CNN tweak here----#
num_filters = [64, 64, 32] #64, 32, 32 up to 70% V, and 64, 64, 32 plateud at ~61-63% after Epoch 1 then increased to 67% | 72 max
batch_size = 30   #smaller batch sizes (less than 50) offer a regularization effect | must be less than 423 bc we are using kfold validation of 5
num_classes = 4
L2 = 0.22 #higher than 0.25 and the validation doesnt reach 70
cnnfilter_stride = [2, 1, 1] #big filter stride makes training to acceptable validation take longer. 
cnnfilter_size = [10, 3, 3] #smaller filter size appears to make higher validation from the beginning. lower than 2 is pretty bad?
cnn_padding = ['same', 'same', 'same']
use_bias = [False, True, False]
use_maxpool = [False, False, False] #not using it makes the model overfit
pool_size = [5, 3, 2]
pool_stride = [2, 1, 1]
use_batchnorm = [True, True, True]
eps = [1e-4, 1e-4, 1e-4]
momentum = [0.3, 0.8, 0.5]
affine = [False, False, False]
dropout = [0.13, 0.21, 0.1, 0.55]

#-----OPTIM tweak here----#
learning_rate = 0.0001 #0 we trained faster with this vs 0.0001 

combo_model = LSTM_FCN.CombinationModel_LSTM_FCN(InputDim, SeqDim, hidden_dim, num_layers, num_filters, cnnfilter_size, cnnfilter_stride, cnn_padding, use_bias, num_classes, use_maxpool, pool_size, pool_stride, use_batchnorm, eps, momentum, affine, dropout)
combo_model.to('cuda:0') #--- activate the GPU


> /content/gdrive/My Drive/Colab Notebooks/ECE_Project/CombinationModel_LSTM_FCN.py(59)__init__()
-> self.FC1 = nn.Linear(self.CNN_Block3.Lout*self.CNN_Block3.out_channels, self.CNN_Block3.Lout)
(Pdb) c


CombinationModel_LSTM_FCN(
  (LSTM): LSTM(22, 32)
  (CNN_Block1): EEG_CNN_BuildingBlock(
    (CNN): Conv1d(22, 64, kernel_size=(10,), stride=(2,), padding=(504,), bias=False)
    (RELU): ReLU()
    (BatchNorm): BatchNorm1d(64, eps=0.0001, momentum=0.3, affine=False, track_running_stats=True)
    (DropOut): Dropout(p=0.13, inplace=False)
  )
  (CNN_Block2): EEG_CNN_BuildingBlock(
    (CNN): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (RELU): ReLU()
    (BatchNorm): BatchNorm1d(64, eps=0.0001, momentum=0.8, affine=False, track_running_stats=True)
    (DropOut): Dropout(p=0.21, inplace=False)
  )
  (CNN_Block3): EEG_CNN_BuildingBlock(
    (CNN): Conv1d(64, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (RELU): ReLU()
    (BatchNorm): BatchNorm1d(32, eps=0.0001, momentum=0.5, affine=False, track_running_stats=True)
    (DropOut): Dropout(p=0.1, inplace=False)
  )
  (FC1): Linear(in_features=32000, out_features=1000, bias=True)
  (DropOutFC): Dropout(p=

In [0]:
########## CHOOSE LOSS  ###########
loss = torch.nn.CrossEntropyLoss()

######### CHOOSE OPTIMIZER ########
optim = torch.optim.Adam(combo_model.parameters(), lr=learning_rate, weight_decay=L2)
#optim = torch.optim.Adadelta(combo_model.parameters()) << does worse than Adam, we overfit so much 

In [10]:
######## TRAIN IT ########
#------ With Early Stopping AND K-Fold Validation -----#
stop_now = False
loss_store = []
validation_store = []
training_store = []
network_store = []
model_store = []

epochs = 1
kfolds = 15
iterations = 1500
iter_ = 0
validate_fold = 0
SEED = 2000

#k fold validation here
np.random.seed(random.randint(1,SEED))
fold_size = int(Trials/kfolds)
idx = np.arange(Trials)
np.random.shuffle(idx)
Xtrain_Shuffled = Xtrain[idx]
Ytrain_Shuffled = Ytrain[idx]

FoldsX = Xtrain_Shuffled.split(fold_size)
FoldsY = Ytrain_Shuffled.split(fold_size)

for epoch in range(epochs):
  if stop_now:
    break

  print(f"...... Training for Epoch {epoch} ......")
  train_correct = 0
  train_total = 0


  for k in range(kfolds): 

    #create training folds by excluding validate fold
    train_folds = list(range(kfolds))
    del train_folds[validate_fold]
    train_folds = np.array(train_folds)

    TrainX = torch.cat([FoldsX[f] for f in train_folds]) 
    TrainY = torch.cat([FoldsY[f] for f in train_folds])
    #create validate fold
    ValidateX = FoldsX[validate_fold]
    ValidateY = FoldsY[validate_fold]

    #initialize the network within the kfold loop
    combo_model = LSTM_FCN.CombinationModel_LSTM_FCN(InputDim, SeqDim, hidden_dim, num_layers, 
                                                     num_filters, cnnfilter_size, cnnfilter_stride, 
                                                     cnn_padding, use_bias, num_classes, use_maxpool, 
                                                     pool_size, pool_stride, use_batchnorm, eps, momentum, 
                                                     affine, dropout)

    combo_model.to('cuda:0') #--- activate the GPU
    optim = torch.optim.Adam(combo_model.parameters(), lr=learning_rate, weight_decay=L2) 
    

    print(f"______ Training for k-folds {train_folds} ______")
    for i in range(iterations):
      #do the batches
      idx = np.arange(TrainX.size(0))
      np.random.shuffle(idx)
      idx = idx[0:batch_size]

      #process the input data
      xtrain = TrainX[idx].view(batch_size, SeqDim, InputDim)
      xtrain = xtrain.to('cuda:0').requires_grad_()
      ytrain = TrainY[idx]

      #put the model in training mode
      combo_model.train(True)

      #forward pass
      optim.zero_grad()
      outFC = combo_model.forward(xtrain.float())
  
      #map to classes
      classes = map_to_class(ytrain.long())
      classes = torch.cat((classes, classes))
      classes = classes.to('cuda:0')

      #backward pass (gradient calculation)
      probs = loss(outFC, classes)
      probs.backward()

      #update weights
      optim.step()

      #update iter counter
      iter_+=1

      #calculate training accuracy
      train_pred = F.softmax(outFC, 1)
      values, expected_classes = torch.max(train_pred, 1)
      train_correct += (1*(expected_classes == classes)).sum()
      train_total += float(classes.size(0))

    print(f"______ Validating for k-fold {k} ______")
    combo_model.eval()
    
    total = 0
    correct = 0

    #process the input data
    idx = np.random.randint(0, Tests, size=batch_size)
    xvalid = ValidateX.view(-1, SeqDim, InputDim)
    xvalid = xvalid.to('cuda:0')
    yvalid = ValidateY

    #forward prop
    predict = combo_model(xvalid.float())
    
    #predict
    pred = F.softmax(predict, 1)
    values, predicted_classes = torch.max(pred, 1)
    expected_classes = map_to_class(yvalid)
    expected_classes = torch.cat((expected_classes, expected_classes))        
    correct += (1*(predicted_classes == expected_classes.to('cuda:0'))).sum()
    total +=float(predicted_classes.size(0))
    
    #evaluate
    validation_accuracy = 100*(correct/total)
    training_accuracy = 100*(train_correct/train_total)

    print(f"correct: {correct}, total: {total}")
    #print(f"prediction: {predicted_classes}\nexpected_classes: {expected_classes}")
    print(f"Iteration: {iter_}, Loss: {probs.item()}, Validation Accuracy: {validation_accuracy}%, Training Accuracy: {training_accuracy}%\n")  

    #store info for graphing later
    validation_store.append(validation_accuracy)
    loss_store.append(probs)
    training_store.append(training_accuracy)

    #store the model
    model_store.append(copy.deepcopy(combo_model))

    #iterate the validate fold
    validate_fold +=1
    
    #--- Early stopping criterion here --- #
    if validation_accuracy > 73.0:
      #store
      print("Early Stopping!")
      stop_now = True
      break
    


...... Training for Epoch 0 ......
> /content/gdrive/My Drive/Colab Notebooks/ECE_Project/CombinationModel_LSTM_FCN.py(59)__init__()
-> self.FC1 = nn.Linear(self.CNN_Block3.Lout*self.CNN_Block3.out_channels, self.CNN_Block3.Lout)
(Pdb) c
______ Training for k-folds [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] ______
______ Validating for k-fold 0 ______
correct: 118, total: 282.0
Iteration: 1500, Loss: 0.6957105398178101, Validation Accuracy: 41.843971252441406%, Training Accuracy: 69.7522201538086%

> /content/gdrive/My Drive/Colab Notebooks/ECE_Project/CombinationModel_LSTM_FCN.py(59)__init__()
-> self.FC1 = nn.Linear(self.CNN_Block3.Lout*self.CNN_Block3.out_channels, self.CNN_Block3.Lout)
(Pdb) n
> /content/gdrive/My Drive/Colab Notebooks/ECE_Project/CombinationModel_LSTM_FCN.py(62)__init__()
-> self.DropOutFC = nn.Dropout(dropout[3])
(Pdb) c
______ Training for k-folds [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14] ______
______ Validating for k-fold 1 ______
correct: 97, total: 282.0


BdbQuit: ignored